In [61]:
from PIL import Image
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense

In [62]:
! mkdir model submit

In [63]:
from os import listdir
from os.path import isfile, isdir, join

def get_imgs_data(mypath, return_file_names=False):
    files = listdir(mypath) # 取得所有檔案與子目錄名稱
    imgs_arr = []
    file_names = []
    for f in files:
        if (not f[0]=='.') and (f[-4:]=='.png'):
            img = Image.open(mypath+f)
            img = img.resize((224, 224), Image.ANTIALIAS)
            imgs_arr.append(np.array(img))
            file_names.append(f[:-4])
            
    op_array =  np.clip(np.array(imgs_arr) / 127.5-1, -1, 1)
    if return_file_names:
        return op_array , file_names
    return op_array 

X_test, ids_test = get_imgs_data("./testing_set/testing_set/", return_file_names=True); print(X_test.shape)
ids_test[:5]

(366, 224, 224, 3)


['003CE5D6A5', '005053493B', '01CBBE0E4C', '02C5C50BFB', '03D9D55867']

In [66]:
classmap = pd.read_csv('classmap.csv', index_col=0, header=None).to_dict()[1]
classmap

{'whippet': 0,
 'Walker_hound': 1,
 'fox_terrier': 2,
 'basset': 3,
 'beagle': 4,
 'papillon': 5,
 'Norfolk_terrier': 6}

In [67]:
X_train_list = [] 
y_train_list = []
for k in classmap.keys():
    c_i =  classmap[k]
    X_train_ = get_imgs_data(f"./training_set/training_set/{k}/"); print(f"class [{c_i}]: ", X_train_.shape)
    X_train_list.append(X_train_)
    y_train_list.append(np.zeros(X_train_.shape[0])+c_i)
X_train = np.concatenate(X_train_list, 0); print(f"X_train: ", X_train.shape)
y_train = np.concatenate(y_train_list, 0); print(f"y_train: ", y_train.shape)
del X_train_list; del y_train_list

class [0]:  (131, 224, 224, 3)
class [1]:  (108, 224, 224, 3)
class [2]:  (110, 224, 224, 3)
class [3]:  (123, 224, 224, 3)
class [4]:  (137, 224, 224, 3)
class [5]:  (138, 224, 224, 3)
class [6]:  (121, 224, 224, 3)
X_train:  (868, 224, 224, 3)
y_train:  (868,)


In [68]:
y_train_categ = to_categorical(y_train); print(f"y_train_categ: ", y_train_categ.shape)

y_train_categ:  (868, 7)


Data Augmentation¶

In [69]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

datagen.fit(X_train)

Build Model

In [70]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization, MaxPool2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras import backend as K
from tensorflow.keras.applications import VGG16, MobileNet, InceptionV3
from tensorflow.keras.layers import Input

In [71]:
# this could also be the output a different Keras model or layer
input_tensor = Input(shape=(224, 224, 3))  # this assumes K.image_data_format() == 'channels_last'
base_model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False)

87916544/87910968 [==============================] - 16s 0us/step


In [72]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(7, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Train

In [73]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

In [74]:
opt = RMSprop(lr=3e-4)
model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy']
             )

In [75]:
epochs = 5
model.fit_generator(datagen.flow(X_train, y_train_categ, batch_size=32),
                    steps_per_epoch=len(X_train) / 32, 
                    epochs=epochs)

Epoch 1/5
28/27 [==============================] - 10s 360ms/step - loss: 1.5777 - acc: 0.5230
Epoch 2/5
28/27 [==============================] - 8s 289ms/step - loss: 0.9137 - acc: 0.6751
Epoch 3/5
28/27 [==============================] - 8s 287ms/step - loss: 0.7775 - acc: 0.7304
Epoch 4/5
28/27 [==============================] - 8s 282ms/step - loss: 0.6396 - acc: 0.7650
Epoch 5/5
28/27 [==============================] - 8s 285ms/step - loss: 0.6265 - acc: 0.7696


In [76]:
# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True

In [77]:
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), 
              loss='categorical_crossentropy',
              metrics=['accuracy']
             )

In [79]:
# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
epochs = 10
his=model.fit_generator(datagen.flow(X_train, y_train_categ, batch_size=32),
                    steps_per_epoch=len(X_train) / 32, 
                    epochs=epochs)

Epoch 1/10
28/27 [==============================] - 9s 329ms/step - loss: 0.3423 - acc: 0.8848
Epoch 2/10
28/27 [==============================] - 12s 416ms/step - loss: 0.3468 - acc: 0.8998
Epoch 3/10
28/27 [==============================] - 11s 397ms/step - loss: 0.3678 - acc: 0.8813
Epoch 4/10
28/27 [==============================] - 11s 386ms/step - loss: 0.3273 - acc: 0.9009
Epoch 5/10
28/27 [==============================] - 12s 413ms/step - loss: 0.3303 - acc: 0.8859
Epoch 6/10
28/27 [==============================] - 11s 406ms/step - loss: 0.3052 - acc: 0.9159
Epoch 7/10
28/27 [==============================] - 12s 429ms/step - loss: 0.2976 - acc: 0.9124
Epoch 8/10
28/27 [==============================] - 14s 495ms/step - loss: 0.3072 - acc: 0.8952
Epoch 9/10
28/27 [==============================] - 13s 455ms/step - loss: 0.3428 - acc: 0.9021
Epoch 10/10
28/27 [==============================] - 15s 547ms/step - loss: 0.3148 - acc: 0.9055


In [80]:
model_name = "CNN_InceptionV3"
postfix = "12"

In [81]:
# save out
from datetime import datetime
y_pred_categ = model.predict(X_test)
y_pred = np.argmax(y_pred_categ, 1)
df_op = pd.DataFrame({'id':ids_test, 'class': y_pred})
t_str = datetime.strftime(datetime.now(), "%Y%m%d%H%M%S")
pth = f'submit/{model_name}_submission_{postfix}_{t_str}.csv'
df_op.to_csv(pth, index=False) # real test score: 0.93
print(f'save scv: {pth}') 
# save out
pth = f'model/{model_name}_model_{postfix}_{t_str}.h5'
model.save_weights(pth)
print(f'save model: {pth}')

save scv: submit/CNN_InceptionV3_submission_12_20190928154149.csv
save model: model/CNN_InceptionV3_model_12_20190928154149.h5


In [82]:
plt.plot(his.history['val_loss'])
plt.xlabel('epochs')
plt.ylabel('validation loss')
plt.show()

KeyError: 'val_loss'

In [ ]:
plt.plot(his.history['acc'])
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.show()

In [ ]:
plt.plot(his.history['acc'],label="Accuracy")
plt.plot(his.history['val_acc'], label="Validation accuracy")
plt.legend()
plt.xlabel('epochs')
plt.show()

In [ ]:
plt.plot(his.history['loss'],label="traing loss")
plt.plot(his.history['val_loss'], label="Validation loss")
plt.legend()
plt.xlabel('epochs')
plt.show()